In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests
from pathlib import Path


: 

In [6]:

pip install selenium


  Using cached selenium-4.25.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.26.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached selenium-4.25.0-py3-none-any.whl (9.7 MB)
Using cached trio-0.26.2-py3-none-any.whl (475 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import ChromiumOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [9]:
def set_chrome_options() -> ChromiumOptions:
    """Sets chrome options for Selenium.Chrome options for headless browser is enabled.
    Args: None
    
    returns:
        Chrome options that can work headless i.e. without actually launching the browser.
    """
    chrome_options = ChromiumOptions()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

In [10]:
def scrape_sitemap(url):
    """
    Extracts all the links from a company's sitemap.xml.
    
    Args:
    url (str): The URL for a company sitemap.xml.
    
    Returns:
    pd.Series: A pandas Series with all the links, or None if no links found.
    """
    try:
        with requests.get(url, headers=headers) as response:
            response.raise_for_status()  # Check if the request was successful
            soup = BeautifulSoup(response.text, 'lxml-xml')
            urls = [link.text.strip() for link in soup.find_all('loc') if link]
            
            if not urls:
                return None  # Return None if no URLs found
            
            extended_urls = []
            for link in urls:
                if link.endswith('xml'):
                    try:
                        with requests.get(link, headers=headers) as response:
                            response.raise_for_status()  # Check if the request was successful
                            nested_soup = BeautifulSoup(response.text, 'lxml')
                            nested_urls = [url.text.strip() for url in nested_soup.find_all('loc') if url]
                            extended_urls.extend(nested_urls)
                    except requests.RequestException as e:
                        print(f"Error occurred while processing {link}: {e}")
                else:
                    extended_urls.append(link)
            
            if not extended_urls:
                return None  # Return None if no extended URLs found
            
            return pd.Series(extended_urls).drop_duplicates().str.strip()
    
    except requests.RequestException as e:
        print(f"Error occurred: {e}")
        return None  # Return None if the initial request fails

In [12]:
pip install spacy

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.2 MB 541.6 kB/s eta 0:00:22
   -- ------------------------------------- 0.8/12.2 MB 671.3 kB/s eta 0:00:18
   --- ------------------------------------ 1.0/12.2 MB 868.0 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/12.2 MB 894.7 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/12.2 MB 894.7 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/12.2 MB 894.7 kB/s eta 0:00:13
   ------ --------------------------------- 1.8/12.2 MB 853.0 kB/s eta 0:00:13
   ------ --------------------------------- 2.1/12.2 MB 939.6 kB/s eta 0:00:11
   ------- -------------------------------- 2.4/12.2 MB 1.0 MB/s eta 0:00:10
   ------- 

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.16 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
langchain-community 0.2.17 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
sagemaker 2.232.2 requires numpy<2.0,>=1.9.0, but you have numpy 2.0.2 which is incompatible.


In [13]:
data = '/C:/Users/kfotiadou/Downloads/sitemap.xml'

In [14]:
data

'/C:/Users/kfotiadou/Downloads/sitemap.xml'

In [15]:
scrape_sitemap(data)

Error occurred: No connection adapters were found for '/C:/Users/kfotiadou/Downloads/sitemap.xml'
